# Prerequisites

1. [Install Docker](http://docs.docker.com/docker-for-windows/install/) on your machine
2. Install a RTSP player such as [VLC Media Player](http://www.videolan.org/vlc/) on your machine
3. [Create Azure container registry](http://docs.microsoft.com/en-us/azure/container-registry/container-registry-get-started-portal) in your Azure subscription (to upload RTSP simulator image for future usage)

# Create local folder for RTSP simulator Dockerfile

In [ ]:
rtspsimLocalFolder = input('Path to directory where RTSP simulator Dockerfile should be created')

import os
if not os.path.exists(rtspsimLocalFolder):
    os.mkdir(rtspsimLocalFolder)

# Write Dockerfile

In [ ]:
%%writefile $rtspsimLocalFolder/Dockerfile

FROM ubuntu:18.04

RUN apt-get update && \
    apt-get upgrade -y && \
    apt-get install -y make build-essential wget libssl-dev

RUN wget http://www.live555.com/liveMedia/public/live555-latest.tar.gz && \
    tar -xzf live555-latest.tar.gz

RUN cd live && \
    ./genMakefiles linux && \
    make

WORKDIR /live/mediaServer

ENTRYPOINT [ "./live555MediaServer" ]

# Create local docker image

In [ ]:
containerImageName = input('RTSP simulator docker image name')

!docker build $rtspsimLocalFolder -t $containerImageName

# Test docker image

To test docker image you can run the following command on your local machine

docker run -p 554:554 -v **localMediaFolder**:/live/mediaServer/media **containerImageName**
    
**localMediaFolder** should point to a folder on your machine where you have media files (e.g. /home/xyz/Videos/) and **containerImageName** should be the image name you provided earlier

# Play Media files

Play media files with URL http://localhost:554/media/my-media-file using RTSP client (e.g. VLC media player) of your choice. Replace my-media-file in this URL with the name of your video file located in your media folder.

Check http://www.live555.com/mediaServer/ for supported file formats


# Login to Azure

In [ ]:
!az login

# Push docker image to Azure container registry

In [ ]:
#Provide name of your Azure container registry (ACR), where you would like to upload the RTSP simulator image
containerRegServiceName = input('Container Registry service name') 

# Tag the local docker image
!docker tag $containerImageName $containerRegServiceName.azurecr.io/$containerImageName

#Push the docker image to ACR
!docker push $containerRegServiceName.azurecr.io/$containerImageName